# GAN Testing

* [YouTube | Generative Adversial Network (GANs) Full Coding Example Tutorial in Tensorflow 2.0!](https://www.youtube.com/watch?v=tX-6CMNnT64)

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0, '../')
import data as d

## Settings

In [22]:
BATCHSIZE = 64
SEQLEN = 100
BUFFER = 10000
EMBED_DIM = 256
EPOCHS = 10
LATENT_UNITS = 1000

## Data Preprocessing

In [4]:
# import The Meditations by Marcus Aurelius
txt = d.meditations()
print(txt[:200])

We have 507 stoic lessons from Marcus Aurelius
From my grandfather Verus I learned good morals and the government of my temper.
From the reputation and remembrance of my father, modesty and a manly character.
From my mother, piety and beneficence,


In [11]:
def format_data(txt):
    vocab = sorted(set(txt))  # create vocab from text string (txt)
    char2idx = {c: i for i, c in enumerate(vocab)}

    data_idx = np.array([char2idx[c] for c in txt])
    
    # max length sequence we can have
    max_seqlen = len(txt) // (SEQLEN)
    
    dataset = tf.data.Dataset.from_tensor_slices(data_idx)
    
    sequences = dataset.batch(SEQLEN + 1, drop_remainder=True)
    
    dataset = sequences.shuffle(BUFFER).batch(BATCHSIZE, drop_remainder=True)
    
    return dataset, char2idx

In [12]:
data, char2idx = format_data(txt)

## Models
Useful Links:
* [Medium | Music generation with Neural Networks](https://medium.com/cindicator/music-generation-with-neural-networks-gan-of-the-week-b66d01e28200)
* [arXiv | C-RNN-GAN: Continuous recurrent neural networks with adversial training](https://arxiv.org/pdf/1611.09904.pdf)
* [Medium | Generating Pokemon-Inspired Musiic from Neural Networks](https://towardsdatascience.com/generating-pokemon-inspired-music-from-neural-networks-bc240014132)

For the loss functions, we will use categorical cross entropy for both as both LSTM and Generative models can use it. This article [Understanding Categorical Cross-Entropy Loss](https://gombru.github.io/2018/05/23/cross_entropy_loss/) covers this type of loss well.

### Generator

In [33]:
def build_generator(latent_units, vocab_size, seqlen, batchsize):
    # dense -> leakyrelu ->  batchnorm * 3 -> reshape to seqlen
    # final dense must be len(vocab_size) which we will then map to chars
    model = tf.keras.Sequential()
    
    # dense -> leakyrelu -> batchnorm
    model.add(tf.keras.layers.Dense(256, input_shape=(latent_units,)))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    
    # dense -> leakyrelu -> batchnorm
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    
    # dense -> leakyrelu -> batchnorm
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    
    # reshape output to seqlen
    model.add(tf.keras.layers.Dense(vocab_size*seqlen, activation='softmax'))
    
    print(model.summary())

    return model    

In [34]:
model_generator = build_generator(LATENT_UNITS, len(char2idx), SEQLEN, BATCHSIZE)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 256)               256256    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 256)               1024      
_________________________________________________________________
dense_23 (Dense)             (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 512)               2048      
_________________________________________________________________
dense_24 (Dense)             (None, 1024)             

### Discriminator

In [17]:
def build_discriminator(vocab_size, embed_dim, batchsize, units):
    model = tf.keras.Sequential()
    # add our embedding layer
    model.add(tf.keras.layers.Embedding(vocab_size, embed_dim,
                                        batch_input_shape=[batchsize, None]))
    # the LSTM layer
    model.add(tf.keras.layers.LSTM(units, return_sequences=True,
                                   stateful=True, dropout=.1))
    # a DNN layer
    model.add(tf.keras.layers.Dense(vocab_size))
    # leaky ReLU activation layer
    model.add(tf.keras.layers.LeakyReLU())

    # final binary classifier layer, 1 or 0
    model.add(tf.keras.layers.Dense(1))
  
    print(model.summary())

    return model

In [18]:
model_discriminator = build_discriminator(len(char2idx), EMBED_DIM, BATCHSIZE, 512)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           17152     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 512)           1574912   
_________________________________________________________________
dense_2 (Dense)              (64, None, 67)            34371     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (64, None, 67)            0         
_________________________________________________________________
dense_3 (Dense)              (64, None, 1)             68        
Total params: 1,626,503
Trainable params: 1,626,503
Non-trainable params: 0
_________________________________________________________________
None
